In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [2]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_c.parquet')

In [4]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2021-05-01') & (df.index <= '2021-08-01')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test


X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

In [10]:
#Create a pool of data

train_pool_c = Pool(X_train_new_c, y_train_new_c)
test_pool_c = Pool(X_test_new_c) 

In [ ]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function" :"LogCosh"
    }

    catboost_model_c = CatBoostRegressor(**params, verbose=400)
    catboost_model_c.fit(train_pool_c)
    pred_c = pd.DataFrame(catboost_model_c.predict(test_pool_c))
    MAE_c = mean_absolute_error(y_test_c, pred_c)
    return MAE_c

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)

[I 2023-11-11 08:18:37,898] A new study created in memory with name: no-name-0d54271e-d2c9-4a01-9fa8-616a6ec70d34


0:	learn: 69.1701711	total: 76.3ms	remaining: 2m 45s
400:	learn: 21.6919578	total: 18.1s	remaining: 1m 19s
800:	learn: 18.9206608	total: 40.8s	remaining: 1m 9s
1200:	learn: 18.1874123	total: 1m 2s	remaining: 50.5s
1600:	learn: 17.6436143	total: 1m 25s	remaining: 30.2s
2000:	learn: 17.0561885	total: 1m 40s	remaining: 8.35s


[I 2023-11-11 08:20:23,327] Trial 0 finished with value: 51.61922475340231 and parameters: {'iterations': 2168, 'learning_rate': 0.007142459071760093, 'depth': 7, 'min_data_in_leaf': 73, 'l2_leaf_reg': 2, 'bagging_temperature': 0.46273908508719064, 'random_strength': 0.9000985661208238, 'border_count': 749, 'rsm': 0.23373756721140937}. Best is trial 0 with value: 51.61922475340231.


2167:	learn: 16.7986986	total: 1m 45s	remaining: 0us
0:	learn: 69.3358464	total: 245ms	remaining: 12m 24s
400:	learn: 38.0233280	total: 1m 50s	remaining: 12m 10s
800:	learn: 24.4912015	total: 3m 42s	remaining: 10m 23s
1200:	learn: 19.1323570	total: 5m 33s	remaining: 8m 32s
1600:	learn: 16.7640812	total: 7m 34s	remaining: 6m 49s
2000:	learn: 15.3228009	total: 9m 27s	remaining: 4m 55s
2400:	learn: 14.3653308	total: 11m 23s	remaining: 3m 3s
2800:	learn: 13.6761732	total: 13m 19s	remaining: 1m 9s
3043:	learn: 13.3014562	total: 14m 27s	remaining: 0us


[I 2023-11-11 08:34:52,668] Trial 1 finished with value: 51.832130599402234 and parameters: {'iterations': 3044, 'learning_rate': 0.0021194842149737, 'depth': 12, 'min_data_in_leaf': 71, 'l2_leaf_reg': 2, 'bagging_temperature': 0.9283436649217105, 'random_strength': 0.7868782124894353, 'border_count': 570, 'rsm': 0.9513351080571983}. Best is trial 0 with value: 51.61922475340231.


...

0:	learn: 69.2739616	total: 675ms	remaining: 28m 59s
400:	learn: 25.1525517	total: 4m 37s	remaining: 25m 5s
800:	learn: 16.2599921	total: 9m 14s	remaining: 20m 29s
1200:	learn: 13.4234099	total: 13m 54s	remaining: 15m 56s
1600:	learn: 11.8951239	total: 18m 57s	remaining: 11m 34s
2000:	learn: 10.8496756	total: 24m 4s	remaining: 6m 56s
2400:	learn: 9.9871905	total: 29m 8s	remaining: 2m 8s
2577:	learn: 9.5828475	total: 31m 25s	remaining: 0us


[I 2023-11-11 09:23:47,799] Trial 13 finished with value: 51.93016767060407 and parameters: {'iterations': 2578, 'learning_rate': 0.0038845662497108266, 'depth': 13, 'min_data_in_leaf': 28, 'l2_leaf_reg': 8, 'bagging_temperature': 0.625019549695535, 'random_strength': 0.5049297490421663, 'border_count': 987, 'rsm': 0.6124464442656238}. Best is trial 10 with value: 50.93136577721838.


0:	learn: 69.4155002	total: 65.8ms	remaining: 22.3s


[I 2023-11-11 09:24:05,608] Trial 14 finished with value: 183.62674491298944 and parameters: {'iterations': 340, 'learning_rate': 0.001156428366466753, 'depth': 6, 'min_data_in_leaf': 5, 'l2_leaf_reg': 5, 'bagging_temperature': 0.9965434481935553, 'random_strength': 0.5312172987202003, 'border_count': 336, 'rsm': 0.3999758382646914}. Best is trial 10 with value: 50.93136577721838.


339:	learn: 53.8926233	total: 17.6s	remaining: 0us
0:	learn: 68.7966060	total: 67.7ms	remaining: 3m 15s
400:	learn: 16.7937204	total: 43.6s	remaining: 4m 31s
800:	learn: 14.6344491	total: 1m 27s	remaining: 3m 49s
1200:	learn: 12.7500964	total: 2m 12s	remaining: 3m 6s
1600:	learn: 11.3188212	total: 2m 57s	remaining: 2m 23s


In [7]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)

with open("optuna-best-parameters_c.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 2004, 'learning_rate': 0.016429110886618033, 'depth': 8, 'colsample_bylevel': 0.7338575760364532, 'min_data_in_leaf': 90, 'l2_leaf_reg': 5, 'has-time': False, 'bagging_temperature': 0.6414768311294344, 'random_strength': 0.4206698081688342, 'border_count': 487, 'rsm': 0.012360675671015169, 'nan_mode': 'min'}
57.13529571086957
